In [15]:
import pandas as pd 
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')


In [16]:
df=df[['v1','v2']]
df.rename(columns={
    'v1': 'Label',
    'v2': 'Text',
    # add more column mappings as needed
}, inplace=True)

In [17]:
import re ##3Used for remooving special words or characters
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mrafi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [5]:
corpus=[]
for i in range(len(df)):
    review = re.sub('[^a-zA-z]',' ', df['Text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english') ]
    review=' '.join(review)
    corpus.append(review)

In [6]:
y = pd.get_dummies(df['Label'],dtype=int)
y= y.iloc[:,0].values

# With BOW

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size=0.2)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, binary=True)
X_train = cv.fit_transform(X_train)
X_test = cv.transform(X_test)

In [9]:
from sklearn.naive_bayes import MultinomialNB
spam_detec_model = MultinomialNB().fit(X_train,y_train)

In [10]:
y_pred = spam_detec_model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_pred,y_test)

0.9802690582959641

In [11]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93       145
           1       0.99      0.99      0.99       970

    accuracy                           0.98      1115
   macro avg       0.95      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115



#  TF-IDF with Lemmatization

In [12]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

"""
POS 
noun-n
verb-v 
adjective - a
adverb - r 
"""

'\nPOS \nnoun-n\nverb-v \nadjective - a\nadverb - r \n'

In [13]:
corpus_tfidf=[]
for i in range(len(df)):
    review = re.sub('[^a-zA-z]',' ', df['Text'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in stopwords.words('english') ]
    review=' '.join(review)
    corpus_tfidf.append(review)

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus_tfidf,y,test_size=0.2)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=100,ngram_range=(2,2))
X_train=tfidf.fit_transform(X_train)
X_test=tfidf.transform(X_test)

In [16]:
from sklearn.naive_bayes import MultinomialNB
spam_detec_model = MultinomialNB().fit(X_train,y_train)
y_pred = spam_detec_model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.9443946188340807
              precision    recall  f1-score   support

           0       0.54      1.00      0.70        72
           1       1.00      0.94      0.97      1043

    accuracy                           0.94      1115
   macro avg       0.77      0.97      0.83      1115
weighted avg       0.97      0.94      0.95      1115



In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=100,ngram_range=(2,2))
X=tfidf.fit_transform(corpus).toarray()

# Word2vec and AvgWord2Vec

In [18]:
import gensim
# import gensim.downloader as api
# wv = api.load("word2vec-google-news-300")
# vec_king = wv["king"]

In [19]:
df

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [32]:
from nltk.stem import WordNetLemmatizer
import re
lemmatizer = WordNetLemmatizer()
words=[]
for i in range(len(df)):
    review = re.sub(r'[\\_)-:1234567890]*','',re.sub('[^a-zA-z]',' ', df['Text'][i]))
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review ] ##Keeping Stopwords
    #review=' '.join(review)
    words.append(review)

In [33]:
for i in range(len(words)):
    if len(words[i])<1:
        print(i)

1611
3374
4822


In [22]:
model = gensim.models.Word2Vec(words,epochs=100)

In [23]:
model.wv.index_to_key

['i',
 'you',
 'to',
 'a',
 'the',
 'u',
 'and',
 'it',
 'in',
 'is',
 'me',
 'my',
 'for',
 'your',
 'call',
 'of',
 's',
 'that',
 'have',
 'on',
 't',
 'now',
 'are',
 'can',
 'm',
 'so',
 'but',
 'not',
 'or',
 'we',
 'do',
 'get',
 'at',
 'be',
 'if',
 'will',
 'ur',
 'with',
 'no',
 'just',
 'this',
 'gt',
 'lt',
 'go',
 'how',
 'up',
 'when',
 'ok',
 'day',
 'what',
 'free',
 'from',
 'all',
 'out',
 'know',
 'll',
 'come',
 'like',
 'time',
 'good',
 'am',
 'then',
 'got',
 'wa',
 'there',
 'he',
 'text',
 'only',
 'love',
 'want',
 'send',
 'p',
 'r',
 'need',
 'txt',
 'one',
 'today',
 'n',
 'going',
 'by',
 'c',
 'don',
 'home',
 'she',
 'about',
 'stop',
 'lor',
 'sorry',
 'k',
 'see',
 'mobile',
 'still',
 'take',
 'back',
 'da',
 'reply',
 'our',
 'think',
 'd',
 'dont',
 'tell',
 'week',
 'phone',
 'hi',
 'new',
 'later',
 'they',
 'her',
 'pls',
 'any',
 'ha',
 'please',
 'co',
 'msg',
 'did',
 'been',
 'min',
 'an',
 'some',
 'make',
 'dear',
 'here',
 'night',
 'who',

In [24]:
model.corpus_count

5572

In [25]:
def avgword2vec(doc):
     
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key], axis=0)

In [26]:
import numpy as np
X=[]
for i in range(len(words)):
    X.append(avgword2vec(words[i]))

c:\Users\mrafi\Documents\LangchainProjects\1-Q&AChatbots\AIProjects\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\mrafi\Documents\LangchainProjects\1-Q&AChatbots\AIProjects\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [27]:
X_new = np.array(X)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5572,) + inhomogeneous part.

In [29]:
X_new.shape

(198, 100)